In [10]:
import threading
import concurrent.futures
import random, time

In [11]:
start_time = time.perf_counter()

def funcToRunInThread(x):
    print(f"(1) Hello from thread {x}!")
    sec = random.randint(1, 5)
    print(f"(2) Thread {x} sleeping for {sec} seconds")
    time.sleep(sec)
    print(f"(3) Goodbye from thread {x}!")

threads = []
for i in range(5):
    my_thread = threading.Thread(target=funcToRunInThread, args=(i+1,))
    my_thread.start()
    threads.append(my_thread)

for thread in threads:
    thread.join()
end_time = time.perf_counter()
print(f"Finished in: {end_time - start_time} seconds")

(1) Hello from thread 1!
(2) Thread 1 sleeping for 4 seconds
(1) Hello from thread 2!
(2) Thread 2 sleeping for 2 seconds
(1) Hello from thread 3!
(2) Thread 3 sleeping for 3 seconds
(1) Hello from thread 4!
(2) Thread 4 sleeping for 4 seconds
(1) Hello from thread 5!
(2) Thread 5 sleeping for 2 seconds
(3) Goodbye from thread 5!(3) Goodbye from thread 2!

(3) Goodbye from thread 3!
(3) Goodbye from thread 1!
(3) Goodbye from thread 4!
Finished in: 4.02958199987188 seconds


In [12]:
def fun1():
    print(f"hi from fun1")
    time.sleep(2)
    print("bye from fun1")

def fun2():
    print(f"hi from fun2")
    time.sleep(5)
    print("bye from fun2")


start_time = time.perf_counter()
fun1()
fun2()
end_time = time.perf_counter()
print(f"Time required for running the functions synchronously: {(end_time - start_time):.2f} seconds")
print('------------------------------------------------------------')

start_time = time.perf_counter()
t1 = threading.Thread(target=fun1)
t2 = threading.Thread(target=fun2)
t1.start()
t2.start()
end_time = time.perf_counter()
print(f"This calculation will run before the threads finish, so it will give wrong value for time \
taken to run the functions: {(end_time - start_time):.2f} seconds")
print('------------------------------------------------------------')

if t1.is_alive():
    print("Thread 1 is still running")
if t2.is_alive():
    print("Thread 2 is still running")

t1.join() # Wait until the thread terminates
print(f"Thread 1 has completed. Time passed: {round(time.perf_counter()-start_time, 2)}")
t2.join()
print(f"Thread 2 has completed. Time passed: {round(time.perf_counter()-start_time, 2)}")
end_time = time.perf_counter()
print(f"Time required for running the functions using threads: {(end_time - start_time):.2f} seconds")
print('------------------------------------------------------------')

hi from fun1
bye from fun1
hi from fun2
bye from fun2
Time required for running the functions synchronously: 7.01 seconds
------------------------------------------------------------
hi from fun1
hi from fun2
This calculation will run before the threads finish, so it will give wrong value for time taken to run the functions: 0.00 seconds
------------------------------------------------------------
Thread 1 is still running
Thread 2 is still running
bye from fun1
Thread 1 has completed. Time passed: 2.02
bye from fun2
Thread 2 has completed. Time passed: 5.02
Time required for running the functions using threads: 5.02 seconds
------------------------------------------------------------


In [13]:
start_time = time.perf_counter()


def fun(seconds):
    print(f"Sleeping for {seconds} seconds")
    time.sleep(seconds)
    return f"Done sleeping for {seconds} seconds..."


with concurrent.futures.ThreadPoolExecutor() as executor:
    f1 = executor.submit(fun, 1) # submit method submits a callable to be executed with the given arguments and returns a future
    print(f1.result()) # Returns the result of the call that the future represents
    print('------------------------------------------------------------')


with concurrent.futures.ThreadPoolExecutor() as executor:
    # secs = [2, 3, 1, 5, 4]
    secs = [3, 2, 5]

    # 1 # Results are returned in the order they are completed
    future_objects = [executor.submit(fun, sec) for sec in secs]
    for future_obj in concurrent.futures.as_completed(future_objects):
        print(future_obj.result())
    print(f"Time passed: {round(time.perf_counter()-start_time, 2)}")
    print('------------------------------------------------------------')

    # 2 # Threads may be completed in any order but the results are returned in the same order as they are started
    future_objects = [executor.submit(fun, sec) for sec in secs]
    for future_obj in future_objects:
        print(future_obj.result())
    print(f"Time passed: {round(time.perf_counter()-start_time, 2)}")
    print('------------------------------------------------------------')

    # 3 # acts similar to #2 # Threads may be completed in any order but the results are returned in the same order as they are started
    results = executor.map(fun, secs)
    for result in results:
        print(result)
    print(f"Time passed: {round(time.perf_counter()-start_time, 2)}")
    print('------------------------------------------------------------')


end_time = time.perf_counter()
print(f"Finished: {round(end_time-start_time, 2)}")

Sleeping for 1 seconds
Done sleeping for 1 seconds...
------------------------------------------------------------
Sleeping for 3 seconds
Sleeping for 2 seconds
Sleeping for 5 seconds
Done sleeping for 2 seconds...
Done sleeping for 3 seconds...
Done sleeping for 5 seconds...
Time passed: 6.05
------------------------------------------------------------
Sleeping for 3 seconds
Sleeping for 2 seconds
Sleeping for 5 seconds
Done sleeping for 3 seconds...
Done sleeping for 2 seconds...
Done sleeping for 5 seconds...
Time passed: 11.06
------------------------------------------------------------
Sleeping for 3 seconds
Sleeping for 2 seconds
Sleeping for 5 seconds
Done sleeping for 3 seconds...
Done sleeping for 2 seconds...
Done sleeping for 5 seconds...
Time passed: 16.06
------------------------------------------------------------
Finished: 16.06


## Use of context managers for concurrent.futures.ThreadPoolExecutor()

In Python, a context manager is an object that defines methods to be used in conjunction with the "with" statement, specifically `__enter__` and `__exit__` methods. The code within the "with" block is the managed context.

In the context of `concurrent.futures.ThreadPoolExecutor()`, the context manager is used to handle the setup and teardown of resources (in this case, threads). When the "with" block is entered, the `ThreadPoolExecutor` is instantiated and its `__enter__` method is called. When the "with" block is exited, the `__exit__` method is called.

The `__exit__` method of the `ThreadPoolExecutor` context manager calls `executor.shutdown(wait=True)`, which will wait until all threads have finished executing before tearing down the `ThreadPoolExecutor`. This ensures that all threads have completed before the program continues, preventing any threads from being cut off prematurely.

So, using `ThreadPoolExecutor` as a context manager in this context ensures that the executor is properly cleaned up when it is no longer needed, and that all threads have finished executing before the program continues. It's a way to manage resources effectively and cleanly in Python.